In [2]:
# 对函数y=2x⊤x关于列向量x求导

import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
x.requires_grad_(True)
#这样就可以在后续查看梯度的值了，默认值是None，要是不设置就看不到梯度取值
x.grad
# 或者可以连起来写
# x = torch.arange(4.0,require_grad=True)

In [5]:
# 计算y

y = 2 * torch.dot(x,x)
y


# torch.dot(x, x) 是向量点积：x⊤x=0^2+1^2+2^2+3^2=0+1+4+9=14
# 再乘以 2：y=2×14=28

# 在深度学习中，一般也都是使用标量求导，也就是进行反向传播的，所以我们可以直接调用y.backward()来计算梯度。

tensor(28., grad_fn=<MulBackward0>)

In [6]:
# 通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
# 验证
x.grad == 4 * x

tensor([True, True, True, True])

In [9]:
# 默认情况下，PyTorch会累积梯度，也就是说每次调用backward()函数时，计算得到的梯度会被加到之前的梯度上，而不是替换掉之前的梯度。
# 因此需要使用.zero_()将梯度清零，以避免梯度累积的影响。
x.grad.zero_()#通过观察注释这一行的输出就知道了
y = x.sum()#将y重新定义为x的元素总和
y.backward()
x.grad

tensor([1., 1., 1., 1.])

非标量变量的反向传播

In [10]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

分离计算

In [11]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [12]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

Python控制流的梯度计算

In [13]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [17]:
# 计算梯度
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [16]:
# 验明梯度是否正确
a.grad == d / a

tensor(True)